In [0]:
%run ../include/setup

In [0]:
import mlflow
from mlflow.pyfunc import load_model

In [0]:
# Load the preprocessed data (parquet file from previous step)
churn_new = spark.read.parquet("/wy/churn_new.parquet")

In [0]:
# Load the pre-trained model from the model registry or file path
modelURL = "models:/" + 'dbacademy.labuser9128531_1739377194.rf2' + "@production"  
predict_churn_udf = mlflow.pyfunc.spark_udf(spark, modelURL, "int") 
spark.udf.register("predict_churn", predict_churn_udf)
model_features = predict_churn_udf.metadata.get_input_schema().input_names()
#start with a delta table
predictions = churn_new.withColumn('churn_prediction', predict_churn_udf(*model_features))
predictions.createOrReplaceTempView("v_churn_prediction")

In [0]:
# Save the predictions to a Delta table
predictions.write.format("delta").mode("overwrite").saveAsTable("churn_prediction2")

In [0]:
# Save the predictions to a file or a database
predictions.write.mode("overwrite").parquet("/wy/predictions.parquet")